# Lab3 - Assignment Sentiment

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook describes the LAB-2 assignment of the Text Mining course. It is about sentiment analysis.

The aims of the assignment are:
* Learn how to run a rule-based sentiment analysis module (VADER)
* Learn how to run a machine learning sentiment analysis module (Scikit-Learn/ Naive Bayes)
* Learn how to run scikit-learn metrics for the quantitative evaluation
* Learn how to perform and interpret a quantitative evaluation of the outcomes of the tools (in terms of Precision, Recall, and F<sub>1</sub>)
* Learn how to evaluate the results qualitatively (by examining the data) 
* Get insight into differences between the two applied methods
* Get insight into the effects of using linguistic preprocessing
* Be able to describe differences between the two methods in terms of their results
* Get insight into issues when applying these methods across different  domains

In this assignment, you are going to create your own gold standard set from 50 tweets. You will the VADER and scikit-learn classifiers to these tweets and evaluate the results by using evaluation metrics and inspecting the data.

We recommend you go through the notebooks in the following order:
* **Read the assignment (see below)**
* **Lab3.2-Sentiment-analysis-with-VADER.ipynb**
* **Lab3.3-Sentiment-analysis.with-scikit-learn.ipynb**
* **Answer the questions of the assignment (see below) using the provided notebooks and submit**

In this assignment you are asked to perform both quantitative evaluations and error analyses:
* a quantitative evaluation concerns the scores (Precision, Recall, and F<sub>1</sub>) provided by scikit's classification_report. It includes the scores per category, as well as micro and macro averages. Discuss whether the scores are balanced or not between the different categories (positive, negative, neutral) and between precision and recall. Discuss the shortcomings (if any) of the classifier based on these scores
* an error analysis regarding the misclassifications of the classifier. It involves going through the texts and trying to understand what has gone wrong. It servers to get insight in what could be done to improve the performance of the classifier. Do you observe patterns in misclassifications?  Discuss why these errors are made and propose ways to solve them.

## Credits
The notebooks in this block have been originally created by [Marten Postma](https://martenpostma.github.io) and [Isa Maks](https://research.vu.nl/en/persons/e-maks). Adaptations were made by [Filip Ilievski](http://ilievski.nl).

## Part I: VADER assignments


### Preparation (nothing to submit):
To be able to answer the VADER questions you need to know how the tool works. 
* Read more about the VADER tool in [this blog](http://t-redactyl.io/blog/2017/04/using-vader-to-handle-sentiment-analysis-with-social-media-text.html).  
* VADER provides 4 scores (positive, negative, neutral, compound). Be sure to understand what they mean and how they are calculated.
* VADER uses rules to handle linguistic phenomena such as negation and intensification. Be sure to understand which rules are used, how they work, and why they are important.
* VADER makes use of a sentiment lexicon. Have a look at the lexicon. Be sure to understand which information can be found there (lemma?, wordform?, part-of-speech?, polarity value?, word meaning?) What do all scores mean? https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vader_lexicon.txt) 


### [3.5 points] Question1:

Regard the following sentences and their output as given by VADER. Regard sentences 1 to 7, and explain the outcome **for each sentence**. Take into account both the rules applied by VADER and the lexicon that is used. You will find that some of the results are reasonable, but others are not. Explain what is going wrong or not when correct and incorrect results are produced. 

```
INPUT SENTENCE 1 I love apples
VADER OUTPUT {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

INPUT SENTENCE 2 I don't love apples
VADER OUTPUT {'neg': 0.627, 'neu': 0.373, 'pos': 0.0, 'compound': -0.5216}

INPUT SENTENCE 3 I love apples :-)
VADER OUTPUT {'neg': 0.0, 'neu': 0.133, 'pos': 0.867, 'compound': 0.7579}

INPUT SENTENCE 4 These houses are ruins
VADER OUTPUT {'neg': 0.492, 'neu': 0.508, 'pos': 0.0, 'compound': -0.4404}

INPUT SENTENCE 5 These houses are certainly not considered ruins
VADER OUTPUT {'neg': 0.0, 'neu': 0.51, 'pos': 0.49, 'compound': 0.5867}

INPUT SENTENCE 6 He lies in the chair in the garden
VADER OUTPUT {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.4215}

INPUT SENTENCE 7 This house is like any house
VADER OUTPUT {'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.3612}
```


- Input 1 : I think that this is correct because the score is above 0.45 for the positive sentiment category. There is a slight neutrality score which seems appropriate.
- Input 2 : This is also correct as there is mainly a negative sentiment score and slight neutrality.
- Input 3 : This is also correct as unlike Input 1 it has a smiley emoji which would mean that it is more positive.
- Input 4 : I think that this should have a more negative sentiment than neutral.
- Input 5 : I think that this is right because "Certainly not" does make it more positive.
- Input 6 : I think that this is wrong because "lies" in this context means laying down and not "lying".So it should be more neutral.
- Input 7 : This could technically be negative because it could be sarcasm.


### [Points: 2.5] Exercise 2: Collecting 50 tweets for evaluation
Collect 50 tweets. Try to find tweets that are interesting for sentiment analysis, e.g., very positive, neutral, and negative tweets. These could be your own tweets (typed in) or collected from the Twitter stream.

We will store the tweets in the file **my_tweets.json** (use a text editor to edit).
For each tweet, you should insert:
* sentiment analysis label: negative | neutral | positive (this you determine yourself, this is not done by a computer)
* the text of the tweet
* the Tweet-URL

from:
```
    "1": {
        "sentiment_label": "",
        "text_of_tweet": "",
        "tweet_url": "",
```
to:
```
"1": {
        "sentiment_label": "positive",
        "text_of_tweet": "All across America people chose to get involved, get engaged and stand up. Each of us can make a difference, and all of us ought to try. So go keep changing the world in 2018.",
        "tweet_url" : "https://twitter.com/BarackObama/status/946775615893655552",
    },
```

You can load your tweets with human annotation in the following way.

In [ ]:
import json

In [ ]:
my_tweets = json.load(open('my_tweets.json'))

In [ ]:
for id_, tweet_info in my_tweets.items():
    print(id_, tweet_info)


### [5 points] Question 3:

Run VADER on your own tweets (see function **run_vader** from notebook **Lab2-Sentiment-analysis-using-VADER.ipynb**). You can use the code snippet below this explanation as a starting point. 
* [2.5 points] a. Perform a quantitative evaluation. Explain the different scores, and explain which scores are most relevant and why.
* [2.5 points] b. Perform an error analysis: select 10 positive, 10 negative and 10 neutral tweets that are not correctly classified and try to understand why. Refer to the VADER-rules and the VADER-lexicon. Of course, if there are less than 10 errors for a category, you only have to check those. For example, if there are only 5 errors for positive tweets, you just describe those.

In [ ]:
import spacy
from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader_model = SentimentIntensityAnalyzer()


In [ ]:
def run_vader(textual_unit,
              lemmatize=False,
              parts_of_speech_to_consider=None,
              verbose=0):
    """
    Run VADER on a sentence from spacy

    :param str textual unit: a textual unit, e.g., sentence, sentences (one string)
    (by looping over doc.sents)
    :param bool lemmatize: If True, provide lemmas to VADER instead of words
    :param set parts_of_speech_to_consider:
    -None or empty set: all parts of speech are provided
    -non-empty set: only these parts of speech are considered.
    :param int verbose: if set to 1, information is printed
    about input and output

    :rtype: dict
    :return: vader output dict
    """
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(textual_unit)

    input_to_vader = []

    for sent in doc.sents:
        for token in sent:

            to_add = token.text

            if lemmatize:
                to_add = token.lemma_

                if to_add == '-PRON-':
                    to_add = token.text

            if parts_of_speech_to_consider:
                if token.pos_ in parts_of_speech_to_consider:
                    input_to_vader.append(to_add)
            else:
                input_to_vader.append(to_add)

    scores = vader_model.polarity_scores(' '.join(input_to_vader))


    return scores

In [ ]:
def vader_output_to_label(vader_output):
    """
    map vader output e.g.,
    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
    to one of the following values:
    a) positive float -> 'positive'
    b) 0.0 -> 'neutral'
    c) negative float -> 'negative'
    
    :param dict vader_output: output dict from vader
    
    :rtype: str
    :return: 'negative' | 'neutral' | 'positive'
    """
    compound = vader_output['compound']
    
    if compound < 0:
        return 'negative'
    elif compound == 0.0:
        return 'neutral'
    elif compound > 0.0:
        return 'positive'
    
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.0}) == 'neutral'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.01}) == 'positive'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': -0.01}) == 'negative'

In [ ]:
tweets = []
all_vader_output = []
gold = []

# settings (to change for different experiments)
to_lemmatize = True 
pos = set()

for id_, tweet_info in my_tweets.items():
    the_tweet = tweet_info['text_of_tweet']
    vader_output = run_vader(the_tweet)
    vader_label = vader_output_to_label(vader_output)# convert vader output to category
    tweets.append(the_tweet)
    all_vader_output.append(vader_label)
    gold.append(tweet_info['sentiment_label'])
    

# use scikit-learn's classification report
from sklearn.metrics import classification_report
print(classification_report(gold, all_vader_output))

Quantitative evaluation:

* Precision: The precision is intuitively the ability of the classifier not to label as positive a sample that is negative. The best value is 1 and the worst value is 0.
*
* Recall: The recall is intuitively the ability of the classifier to find all the positive samples. The best value is 1 and the worst value is 0.

* F1-score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal.

* Support: The support is the number of occurrences of each class in y_true.

* Accuracy: The accuracy is the number of correctly classified samples divided by the total number of samples. The best value is 1 and the worst value is 0.

* Macro avg: Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

* Weighted avg: Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that is not between precision and recall.

* Micro avg: Calculate metrics globally by counting the total true positives, false negatives and false positives. This is a better metric when we have class imbalance.

* Samples avg: Calculate metrics for each instance, and find their average (only meaningful for multilabel classification where this differs from accuracy_score).
*

In [ ]:
# error analysis
misclassified_pos = []
misclassified_neg = []
misclassified_neu = []

for i, (tweet, vader_label, gold_label) in enumerate(zip(tweets, all_vader_output, gold)):
    if vader_label != gold_label:
        if gold_label == 'positive':
            misclassified_pos.append((i, tweet, vader_label, gold_label))
        elif gold_label == 'negative':
            misclassified_neg.append((i, tweet, vader_label, gold_label))
        elif gold_label == 'neutral':
            misclassified_neu.append((i, tweet, vader_label, gold_label))

print('Number of misclassified positive tweets: {}'.format(len(misclassified_pos)))
print('Number of misclassified negative tweets: {}'.format(len(misclassified_neg)))
print('Number of misclassified neutral tweets: {}'.format(len(misclassified_neu)))


In [ ]:
# print misclassified positive tweets
for i, tweet, vader_label, gold_label in misclassified_pos:
    print('Tweet: {}'.format(tweet))
    print('Vader label: {}'.format(vader_label))
    print('Gold label: {}'.format(gold_label))
    print('-----------------------------')

Error Analysis on Positive Tweets:


In [ ]:
# print misclassified negative tweets
for i, tweet, vader_label, gold_label in misclassified_neg:
    print('Tweet: {}'.format(tweet))
    print('Vader label: {}'.format(vader_label))
    print('Gold label: {}'.format(gold_label))
    print('-----------------------------')


Error Analysis on Negative Tweets:

In [ ]:
# print misclassified neutral tweets
for i, tweet, vader_label, gold_label in misclassified_neu:
    print('Tweet: {}'.format(tweet))
    print('Vader label: {}'.format(vader_label))
    print('Gold label: {}'.format(gold_label))
    print('-----------------------------')

Error Analysis on Neutral Tweets:

### [4 points] Question 4:
Run VADER on the set of airline tweets with the following settings:

* Run VADER (as it is) on the set of airline tweets 
* Run VADER on the set of airline tweets after having lemmatized the text
* Run VADER on the set of airline tweets with only adjectives
* Run VADER on the set of airline tweets with only adjectives and after having lemmatized the text
* Run VADER on the set of airline tweets with only nouns
* Run VADER on the set of airline tweets with only nouns and after having lemmatized the text
* Run VADER on the set of airline tweets with only verbs
* Run VADER on the set of airline tweets with only verbs and after having lemmatized the text

* [1 point] a. Generate for all separate experiments the classification report, i.e., Precision, Recall, and F<sub>1</sub> scores per category as well as micro and macro averages. **Use a different code cell (or multiple code cells) for each experiment.**
* [3 points] b. Compare the scores and explain what they tell you.
* - Does lemmatisation help? Explain why or why not.
* - Are all parts of speech equally important for sentiment analysis? Explain why or why not.

In [ ]:
import pathlib
from sklearn.datasets import load_files
cwd = pathlib.Path.cwd()
airline_tweets_folder = cwd.joinpath('airlinetweets')
airline_tweets_train = load_files(str(airline_tweets_folder))


In [ ]:
# run vader on the set of airline tweets
tweets = []
all_vader_output = []
gold = []

for i, tweet in enumerate(airline_tweets_train.data[:100]):
    tweets.append(tweet.decode('utf-8'))
    vader_output = run_vader(tweet.decode('utf-8'))
    vader_label = vader_output_to_label(vader_output)
    all_vader_output.append(vader_label)
    gold.append(airline_tweets_train.target_names[airline_tweets_train.target[i]])

from sklearn.metrics import classification_report
print("VADER (as it is) on the set of airline tweets Classification Report")
print(classification_report(gold, all_vader_output))

In [ ]:
# run vader on the set of airline tweets after having lemmatized the text
all_vader_output = []
gold = []

for i, tweet in enumerate(airline_tweets_train.data[:100]):
    vader_output = run_vader(tweet.decode('utf-8'), lemmatize=True)
    vader_label = vader_output_to_label(vader_output)
    all_vader_output.append(vader_label)
    gold.append(airline_tweets_train.target_names[airline_tweets_train.target[i]])


print("VADER on the set of airline tweets after having lemmatized the text Classification Report")
print(classification_report(gold, all_vader_output))

KeyboardInterrupt: 

In [ ]:
# run vader on the set of airline tweets with only adjectives
all_vader_output = []
gold = []

for i, tweet in enumerate(airline_tweets_train.data[:100]):
    vader_output = run_vader(tweet.decode('utf-8'), parts_of_speech_to_consider={'ADJ'})
    vader_label = vader_output_to_label(vader_output)
    all_vader_output.append(vader_label)
    gold.append(airline_tweets_train.target_names[airline_tweets_train.target[i]])


print("VADER on the set of airline tweets with only adjectives Classification Report")
print(classification_report(gold, all_vader_output))

In [ ]:
# run vader on the set of airline tweets with only adjectives and after having lemmatized the text
all_vader_output = []
gold = []

for i, tweet in enumerate(airline_tweets_train.data[:100]):
    vader_output = run_vader(tweet.decode('utf-8'), parts_of_speech_to_consider={'ADJ'}, lemmatize=True)
    vader_label = vader_output_to_label(vader_output)
    all_vader_output.append(vader_label)
    gold.append(airline_tweets_train.target_names[airline_tweets_train.target[i]])


print("VADER on the set of airline tweets with only adjectives and after having lemmatized the text Classification Report")
print(classification_report(gold, all_vader_output))

In [ ]:
# run vader on the set of airline tweets with only nouns
all_vader_output = []
gold = []

for i, tweet in enumerate(airline_tweets_train.data[:100]):
    vader_output = run_vader(tweet.decode('utf-8'), parts_of_speech_to_consider={'NOUN'})
    vader_label = vader_output_to_label(vader_output)
    all_vader_output.append(vader_label)
    gold.append(airline_tweets_train.target_names[airline_tweets_train.target[i]])


print("VADER on the set of airline tweets with only nouns Classification Report")
print(classification_report(gold, all_vader_output))


In [ ]:
# run vader on the set of airline tweets with only nouns and after having lemmatized the text
all_vader_output = []
gold = []

for i, tweet in enumerate(airline_tweets_train.data[:100]):
    vader_output = run_vader(tweet.decode('utf-8'), parts_of_speech_to_consider={'NOUN'}, lemmatize=True)
    vader_label = vader_output_to_label(vader_output)
    all_vader_output.append(vader_label)
    gold.append(airline_tweets_train.target_names[airline_tweets_train.target[i]])


print("VADER on the set of airline tweets with only nouns and after having lemmatized the text Classification Report")
print(classification_report(gold, all_vader_output))


In [ ]:
# run vader on the set of airline tweets with only verbs
all_vader_output = []
gold = []

for i, tweet in enumerate(airline_tweets_train.data[:100]):
    vader_output = run_vader(tweet.decode('utf-8'), parts_of_speech_to_consider={'VERB'})
    vader_label = vader_output_to_label(vader_output)
    all_vader_output.append(vader_label)
    gold.append(airline_tweets_train.target_names[airline_tweets_train.target[i]])


print("VADER on the set of airline tweets with only verbs Classification Report")
print(classification_report(gold, all_vader_output))

In [ ]:
# run vader on the set of airline tweets with only verbs and after having lemmatized the text
all_vader_output = []
gold = []

for i, tweet in enumerate(airline_tweets_train.data[:100]):
    vader_output = run_vader(tweet.decode('utf-8'), parts_of_speech_to_consider={'VERB'}, lemmatize=True)
    vader_label = vader_output_to_label(vader_output)
    all_vader_output.append(vader_label)
    gold.append(airline_tweets_train.target_names[airline_tweets_train.target[i]])


print("VADER on the set of airline tweets with only verbs and after having lemmatized the text Classification Report")
print(classification_report(gold, all_vader_output))

b. Compare the scores and explain what they tell you.
* - Does lemmatisation help? Explain why or why not.
* - Are all parts of speech equally important for sentiment analysis? Explain why or why not.

## Part II: scikit-learn assignments
### [4 points] Question 5
Train the scikit-learn classifier (Naive Bayes) using the airline tweets.

+ Train the model on the airline tweets with 80% training and 20% test set and default settings (TF-IDF representation, min_df=2)
+ Train with different settings:
    + with respect to vectorizing: TF-IDF ('airline_tfidf') vs. Bag of words representation ('airline_count') 
    + with respect to the frequency threshold (min_df). Carry out experiments with increasing values for document frequency (min_df = 2; min_df = 5; min_df =10) 
* [1 point] a. Generate a classification_report for all experiments
* [3 points] b. Look at the results of the experiments with the different settings and try to explain why they differ: 
    + which category performs best, is this the case for any setting?
    + does the frequency threshold affect the scores? Why or why not according to you?

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

airline_vec = CountVectorizer(min_df=2, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english')) # stopwords are removed
#  bag of words representation of the airline tweets
airline_counts = airline_vec.fit_transform(airline_tweets_train.data)

docs_train, docs_test, y_train, y_test = train_test_split(
    airline_counts, # the tf-idf model
    airline_tweets_train.target, # the category values for each tweet
    test_size = 0.20 # we use 80% for training and 20% for development
    )

clf = MultinomialNB().fit(docs_train, y_train)
y_pred = clf.predict(docs_test)

print("Classification report for the Naive Bayes classifier on the airline tweets with 80% training and 20% test set and default settings (TF-IDF representation, min_df=2)")
print(classification_report(y_test, y_pred, target_names=airline_tweets_train.target_names))

In [ ]:
# TF-IDF representation of the airline tweets
tfidf_transformer = TfidfTransformer()
airline_tfidf = tfidf_transformer.fit_transform(airline_counts)
docs_train2, docs_test2, y_train2, y_test2 = train_test_split(
    airline_tfidf, # the tf-idf model
    airline_tweets_train.target, # the category values for each tweet
    test_size = 0.20 # we use 80% for training and 20% for development
    )
clf2 = MultinomialNB().fit(docs_train2, y_train2)
y_pred2 = clf2.predict(docs_test2)

print("Classification report for the Naive Bayes classifier on the airline tweets with 80% training and 20% test set and default settings (TF-IDF representation, min_df=2)")
print(classification_report(y_test2, y_pred2, target_names=airline_tweets_train.target_names))

In [ ]:
# TF-IDF representation of the airline tweets with min_df=5
airline_vec = CountVectorizer(min_df=5, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english')) # stopwords are removed
#  bag of words representation of the airline tweets
airline_counts = airline_vec.fit_transform(airline_tweets_train.data)

tfidf_transformer = TfidfTransformer()
airline_tfidf = tfidf_transformer.fit_transform(airline_counts)
docs_train3, docs_test3, y_train3, y_test3 = train_test_split(
    airline_tfidf, # the tf-idf model
    airline_tweets_train.target, # the category values for each tweet
    test_size = 0.20 # we use 80% for training and 20% for development
    )
clf3 = MultinomialNB().fit(docs_train3, y_train3)
y_pred3 = clf3.predict(docs_test3)

print("Classification report for the Naive Bayes classifier on the airline tweets with 80% training and 20% test set and default settings (TF-IDF representation, min_df=5)")
print(classification_report(y_test3, y_pred3, target_names=airline_tweets_train.target_names))

In [ ]:
# TF-IDF representation of the airline tweets with min_df=10
airline_vec = CountVectorizer(min_df=10, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english')) # stopwords are removed
#  bag of words representation of the airline tweets
airline_counts = airline_vec.fit_transform(airline_tweets_train.data)

tfidf_transformer = TfidfTransformer()
airline_tfidf = tfidf_transformer.fit_transform(airline_counts)
docs_train4, docs_test4, y_train4, y_test4 = train_test_split(
    airline_tfidf, # the tf-idf model
    airline_tweets_train.target, # the category values for each tweet
    test_size = 0.20 # we use 80% for training and 20% for development
    )
clf4 = MultinomialNB().fit(docs_train4, y_train4)
y_pred4 = clf4.predict(docs_test4)

print("Classification report for the Naive Bayes classifier on the airline tweets with 80% training and 20% test set and default settings (TF-IDF representation, min_df=10)")
print(classification_report(y_test4, y_pred4, target_names=airline_tweets_train.target_names))


* [3 points] b. Look at the results of the experiments with the different settings and try to explain why they differ:
    + which category performs best, is this the case for any setting?
    + does the frequency threshold affect the scores? Why or why not according to you?

Answer:

### [4 points] Question 6: Inspecting the best scoring features 

+ Train the scikit-learn classifier (Naive Bayes) model with the following settings (airline tweets 80% training and 20% test;  Bag of words representation ('airline_count'), min_df=2)
* [1 point] a. Generate the list of best scoring features per class (see function **important_features_per_class** below) [1 point]
* [3 points] b. Look at the lists and consider the following issues: 
    + [1 point] Which features did you expect for each separate class and why?
    + [1 point] Which features did you not expect and why ? 
    + [1 point] The list contains all kinds of words such as names of airlines, punctuation, numbers and content words (e.g., 'delay' and 'bad'). Which words would you remove or keep when trying to improve the model and why? 

In [ ]:
def important_features_per_class(vectorizer,classifier,n=80):
    class_labels = classifier.classes_
    feature_names =vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]
    topn_class2 = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_class3 = sorted(zip(classifier.feature_count_[2], feature_names),reverse=True)[:n]
    print("Important words in negative documents")
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
    print("-----------------------------------------")
    print("Important words in neutral documents")
    for coef, feat in topn_class2:
        print(class_labels[1], coef, feat) 
    print("-----------------------------------------")
    print("Important words in positive documents")
    for coef, feat in topn_class3:
        print(class_labels[2], coef, feat) 

# example of how to call from notebook:

airline_vec = CountVectorizer(min_df=2, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english')) # stopwords are removed
#  bag of words representation of the airline tweets
airline_counts = airline_vec.fit_transform(airline_tweets_train.data)

docs_train, docs_test, y_train, y_test = train_test_split(
    airline_counts, # the bag of words model
    airline_tweets_train.target, # the category values for each tweet
    test_size = 0.20 # we use 80% for training and 20% for development
    )

clf = MultinomialNB().fit(docs_train, y_train)
y_pred = clf.predict(docs_test)
important_features_per_class(airline_vec, clf)

### [Optional! (will not  be graded)] Question 7
Train the model on airline tweets and test it on your own set of tweets
+ Train the model with the following settings (airline tweets 80% training and 20% test;  Bag of words representation ('airline_count'), min_df=2)
+ Apply the model on your own set of tweets and generate the classification report
* [1 point] a. Carry out a quantitative analysis.
* [1 point] b. Carry out an error analysis on 10 correctly and 10 incorrectly classified tweets and discuss them
* [2 points] c. Compare the results (cf. classification report) with the results obtained by VADER on the same tweets and discuss the differences.

### [Optional! (will not be graded)] Question 8: trying to improve the model
* [2 points] a. Think of some ways to improve the scikit-learn Naive Bayes model by playing with the settings or applying linguistic preprocessing (e.g., by filtering on part-of-speech, or removing punctuation). Do not change the classifier but continue using the Naive Bayes classifier. Explain what the effects might be of these other settings 
+ [1 point] b. Apply the model with at least one new setting (train on the airline tweets using 80% training, 20% test) and generate the scores
* [1 point] c. Discuss whether the model achieved what you expected.

## End of this notebook